### Install Dependencies

In [2]:
import numpy as np
import timeit

### Import files

In [3]:
standard_factorizations_path = "/Users/kev/Documents/projects/alphatensor-analysis/algorithms/factorizations_r.npz"
mod2_factorizations_path = "/Users/kev/Documents/projects/alphatensor-analysis/algorithms/factorizations_f2.npz"

standard_factorizations = dict(np.load(standard_factorizations_path, allow_pickle=True))

mod2_factorizations = dict(np.load(mod2_factorizations_path, allow_pickle=True))

In [5]:
# Print available factorizations and their shapes.
for key in mod2_factorizations:
  naive_u, v, w = mod2_factorizations[key]
  rank = naive_u.shape[-1]
  assert rank == v.shape[-1] and rank == w.shape[-1]
  print(f'{key}: rank={naive_u.shape[-1]}')

2,2,2: rank=7
2,2,3: rank=11
2,2,4: rank=14
2,2,5: rank=18
2,3,3: rank=15
2,3,4: rank=20
2,3,5: rank=25
2,4,4: rank=26
2,4,5: rank=33
2,5,5: rank=40
3,3,3: rank=23
3,3,4: rank=29
3,3,5: rank=36
3,4,4: rank=38
3,4,5: rank=47
3,5,5: rank=58
4,4,4: rank=47
4,4,5: rank=63
4,5,5: rank=76
5,5,5: rank=96


### Generate Naive Algorithm Factorization

In [ ]:
# takes n,m,p dimensions of two matrices multiplied together
# returns numpy.ndarray
def generate_naive_factorization(n: int ,m: int, p:int):

    multiplications = n * m *p
    u = np.zeros(((n*m),multiplications ))
    v = np.zeros(((m*p),multiplications ))
    w = np.zeros(((n*p),multiplications ))
    
    u_flat = u.flatten()

    
    multiplication_count =0
    c_index =0
    #loop over matrix c row
    for i in range(0,n):
        for z in range(0,p):
            for x in range(0,int(multiplications/(n*p))):
                u[(i*m)+(x)][multiplication_count] = 1
                v[z+(x*p)][multiplication_count]=1
                w[c_index][multiplication_count] = 1

                multiplication_count = multiplication_count + 1
            c_index = c_index + 1
    return(u,v,w)

(array([[1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 1.]]), array([[1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1.]]), array([[1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1.]]))


### Multiply Matrices

In [ ]:
def matrix_multiplication_using_factorization(matrix_a, matrix_b, u,v,w):
  """Multiplies two matrices using the provided factorization.

  Args:
    matrix_a: The first matrix (numpy array).
    matrix_b: The second matrix (numpy array).
    u,v,w: numpy array matrix factorizations

  Returns:
    The product of matrix_a and matrix_b.
  """

  a_flat = matrix_a.flatten()
  b_flat = matrix_b.flatten()

  # Perform the matrix multiplication using the factorization
  intermediate_a = np.dot(a_flat, u)
  intermediate_b = np.dot(b_flat, v)
  intermediate = (intermediate_a * intermediate_b)
  result_flat = np.dot( w,intermediate)

  # Reshape the result to the correct matrix dimensions
  rows_a = matrix_a.shape[0]
  cols_b = matrix_b.shape[1]
  result_matrix = result_flat.reshape((rows_a, cols_b), order='f')

  return result_matrix



#np.allclose(product_matrix, standard_product)


Timing functions

In [143]:
matrix_a = np.random.rand(2, 2)
matrix_b = np.random.rand(2, 2)
(naive_u,naive_v,naive_w) = generate_naive_factorization(2,2,2)
# Assuming you have a factorization for matrices of size (3, 4, 5)
factorization_key = '2,2,2'
(u,v,w) = standard_factorizations[factorization_key]


In [144]:

execution_time = timeit.timeit(
    stmt='matrix_multiplication_using_factorization(matrix_a, matrix_b, u,v,w)',
    number=10000,  
    globals=globals()
)


print(f"Average execution time for google over 10000 runs: {execution_time / 1000} seconds")


Average execution time for google over 10000 runs: 6.273499999952036e-05 seconds


In [145]:
execution_time_naive = timeit.timeit(
    stmt='matrix_multiplication_using_factorization(matrix_a, matrix_b, naive_u,naive_v,naive_w)',
    number=10000,  
    globals=globals()
)
print(f"Average execution time for naive over 10000 runs: {execution_time_naive / 1000} seconds")

Average execution time for naive over 10000 runs: 5.008799999995972e-05 seconds


In [148]:
def _get_2x2x2_strassen() -> np.ndarray:
  """Returns [3, 4, 7] array, representing a rank-7 factorization of T_2."""

  # List of 7 factors, each of shape [3, 4].
  factors = [[[1, 0, 0, 1], [1, 0, 0, 1], [1, 0, 0, 1]],
             [[1, 0, 0, 0], [0, 1, 0, -1], [0, 0, 1, 1]],
             [[0, 1, 0, -1], [0, 0, 1, 1], [1, 0, 0, 0]],
             [[0, 0, 1, 1], [1, 0, 0, 0], [0, 1, 0, -1]],
             [[0, 0, 0, 1], [-1, 0, 1, 0], [1, 1, 0, 0]],
             [[-1, 0, 1, 0], [1, 1, 0, 0], [0, 0, 0, 1]],
             [[1, 1, 0, 0], [0, 0, 0, 1], [-1, 0, 1, 0]]]

  # Transpose into our standard format [3, S, R] = [3, 4, 7],
  return np.transpose(np.array(factors, dtype=np.int32), [1, 2, 0])

print(_get_2x2x2_strassen())

print(standard_factorizations["2,2,2"])

[[[ 1  1  0  0  0 -1  1]
  [ 0  0  1  0  0  0  1]
  [ 0  0  0  1  0  1  0]
  [ 1  0 -1  1  1  0  0]]

 [[ 1  0  0  1 -1  1  0]
  [ 0  1  0  0  0  1  0]
  [ 0  0  1  0  1  0  0]
  [ 1 -1  1  0  0  0  1]]

 [[ 1  0  1  0  1  0 -1]
  [ 0  0  0  1  1  0  0]
  [ 0  1  0  0  0  0  1]
  [ 1  1  0 -1  0  1  0]]]
[[[ 0  1  1  0  1  1  0]
  [ 0  0 -1  1  0  0  0]
  [ 1  1  1  0  1  0  0]
  [-1 -1 -1  0  0  0  1]]

 [[ 0  0  0  0  1  1  0]
  [ 1  1  0  0  1  0  1]
  [ 0  1  1  1  1  0  0]
  [ 0  1  1  0  1  0  1]]

 [[ 0  0  0  1  0  1  0]
  [ 0 -1  0  0  1 -1 -1]
  [-1  1 -1 -1  0  0  0]
  [ 1  0  0  0  0  0  1]]]
